Ad hoc concordance analyses of the Lx results. Lx results was using a previous version of analyze results pipeline that didn't have concordance yet; this adds it in

In [7]:
import glob
import os
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
def getConcordance(df, threshold=-0.6):
    df['concordance'] = 0
    df.loc[(df.y_actual <= threshold) & (df.y_pred <= threshold), 'concordance'] = 1
    df.loc[(df.y_actual > threshold) & (df.y_pred > threshold), 'concordance'] = 1
    return sum(df.concordance == 1) / len(df)

def genConcordance(Lx_dir):
    outdir_concord = '%s/anlyz/concordance/' % Lx_dir
    if not os.path.exists(outdir_concord):
        os.makedirs(outdir_concord)

    y_compr_fnames = glob.glob(os.path.join(Lx_dir, 'model_perf', 'y_compr_*.pkl'))
    if len(y_compr_fnames) > 0:
        df_conc_tr = pd.DataFrame()
        df_conc_te = pd.DataFrame()
        for fname in y_compr_fnames:
            f = re.sub('.*_compr_', '', fname)
            gene = re.sub('\.pkl', '', f)
            df = pickle.load(open(fname, 'rb'))

            tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df['tr'])}])
            df_conc_tr = pd.concat([df_conc_tr, tmp])

            tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df['te'])}])
            df_conc_te = pd.concat([df_conc_te, tmp])

        df_conc_tr.to_csv('%s/concordance_tr.csv' % outdir_concord, index=False)
        df_conc_te.to_csv('%s/concordance_te.csv' % outdir_concord, index=False)

        plt.figure()
        ax = sns.distplot(df_conc_tr.concordance)
        ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
        plt.savefig("%s/concordance_tr.pdf" % outdir_concord)
        plt.close()

        plt.figure()
        ax = sns.distplot(df_conc_te.concordance)
        ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
        plt.savefig("%s/concordance_te.pdf" % outdir_concord)
        plt.close()

In [9]:
for fdir in glob.glob('../out/20.0518 Lx/*/'):
    print(fdir)
    genConcordance(fdir)

../out/20.0518 Lx/L100_reg_rf_boruta/
../out/20.0518 Lx/L200only_reg_lm/
../out/20.0518 Lx/L200only_reg_rf_boruta/
../out/20.0518 Lx/L300only_reg_rf_boruta/
../out/20.0518 Lx/L100only_reg_rf_boruta/
../out/20.0518 Lx/L100only_reg_elasticnet_all/
../out/20.0518 Lx/L75only_reg_elasticnet_all/
../out/20.0518 Lx/L25only_reg_rf_boruta/
../out/20.0518 Lx/L100only_reg_lm/
../out/20.0518 Lx/L75only_reg_rf_boruta/
../out/20.0518 Lx/L200only_reg_elasticnet_all/
../out/20.0518 Lx/L100only_reg_elasticnet/
../out/20.0518 Lx/L25only_reg_elasticnet_all/
../out/20.0518 Lx/L100only_reg_rf_boruta_all/
